# Clickhouse Stats


----
## Connection
Connection parameters are defined in the ./jupyter/connections.ini

In [6]:
%load_ext sql
%config SqlMagic.displaylimit = 50
%sql --section clickhouse

The sql extension is already loaded. To reload it, use:
  %reload_ext sql


Connecting to 'clickhouse'

----
## Database Selection

In [13]:
database = "demo"
%sql use {{database}}

Running query in 'clickhouse'

++
||
++
++

----
## Database Compression Ratio

In [28]:
%%sql 
SELECT 
    SUM(total_rows) as Rows,
    FLOOR(SUM(total_bytes)/1024/1024, 2) as Comp_in_mb,
    FLOOR(SUM(total_bytes_uncompressed)/1024/1024, 2) as Uncomp_in_mb,
    FLOOR(Uncomp_in_mb/Comp_in_mb,2) as Comp_ratio
FROM `system`.tables 
WHERE 
	database = '{{database}}' 

Running query in 'clickhouse'

Rows,Comp_in_mb,Uncomp_in_mb,Comp_ratio
8795488,360.85,1580.19,4.37


----
## Table Statistics

In [19]:
%%sql SELECT 
    name,
    total_rows,
    FLOOR(total_bytes/1024/1024, 2) as compressed_in_mb,
    FLOOR(total_bytes_uncompressed/1024/1024, 2) as uncompressed_in_mb,
    FLOOR(total_bytes_uncompressed / total_bytes, 2) AS compression_ratio
FROM
    `system`.tables
WHERE
    database = '{{database}}' AND 
    total_rows > 0
ORDER BY
    total_rows DESC

Running query in 'clickhouse'

name,total_rows,compressed_in_mb,uncompressed_in_mb,compression_ratio
timeline_short,3707699,52.56,124.09,2.36
hive,3598227,168.61,706.59,4.19
ntfs_info,1380834,134.04,722.78,5.39
srum_application_resources,49920,2.62,10.95,4.17
srum_energy_estimation,36512,1.69,7.44,4.38
evtx,9596,0.84,7.25,8.62
srum_network_data_usage,8528,0.25,1.19,4.6
srum_vfuprov,1912,0.06,0.27,3.94
srum_energy_usage,804,0.02,0.08,2.82
srum_network_connectivity_usage,744,0.02,0.06,2.73


----
## Timeline data usage

In [29]:
%%sql 
SELECT 
	SUM(total_bytes) as total_bytes,
	(SELECT total_bytes FROM `system`.tables WHERE database = '{{database}}' AND name='timeline_short') as timeline_bytes,
	floor((SELECT total_bytes FROM `system`.tables WHERE database = '{{database}}' AND name='timeline_short')/total_bytes,2) as timeline_to_data_ratio
FROM `system`.tables 
WHERE 
	database = '{{database}}' AND
	name != 'timeline_short' AND
	total_rows>0

Running query in 'clickhouse'

total_bytes,timeline_bytes,timeline_to_data_ratio
323254138,55092754,0.17
